<a href="https://colab.research.google.com/github/jigjid/github_task/blob/main/Credit_information_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# [Question 1] Confirmation of the content of the competition
1.what to learn and what to predict
In data science and machine learning, the goal is typically to learn patterns and relationships within a dataset, and then use this knowledge to make predictions on new, unseen data. So, what to learn and what to predict will depend on the specific problem you are trying to solve. For example, if you are working on a classification problem, you will learn to differentiate between different classes in the data, and then predict which class new data points belong to. If you are working on a regression problem, you will learn to predict a numerical value based on the input features.

2.What kind of file to create and submit to Kaggle
Kaggle is a platform for data science competitions, and typically requires participants to submit a specific kind of file format for their predictions. The specific format will depend on the competition, but often involves submitting a CSV file with predictions for a test dataset. It's important to carefully read the competition rules and instructions to make sure you are submitting the correct file type and format.

3.What kind of index value will the submitted work be evaluated by?
The evaluation metric for a Kaggle competition will also depend on the specific problem being solved. Typically, the competition organizers will specify a specific evaluation metric that will be used to rank submissions and determine the winner. This could be a classification accuracy metric, mean squared error, or some other metric that is appropriate for the problem at hand. It's important to pay close attention to the evaluation metric, as optimizing for it can be crucial to achieving a high score and ranking well in the competition.

## [Question 2] Learning and verification

In [21]:
import numpy as np
import pandas as pd
app_train = pd.read_csv('application_train.csv')

In [22]:
""
def learn_ver(X, y):
  from sklearn.model_selection import train_test_split
  from sklearn.preprocessing import StandardScaler
  from sklearn.linear_model import SGDRegressor
  from sklearn.metrics import mean_squared_error
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123) 
  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train_std = scaler.transform(X_train)
  X_test_std = scaler.transform(X_test)
  reg = SGDRegressor()
  reg.fit(X_train_std, y_train)
  y_pred = reg.predict(X_test_std)
  mse = mean_squared_error(y_test,y_pred)
  rmse = np.sqrt(mse)
  return  mse, rmse 
     

In [23]:
train = app_train[['SK_ID_CURR']].values
target = app_train[['TARGET']].values
learn_ver(X=train, y=target )

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(0.0759946936752337, 0.27567135084232763)

In [24]:
train = app_train[['DAYS_BIRTH']].values
learn_ver(X=train, y=target )

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(0.07536017701450481, 0.27451808139811995)

In [25]:

train = app_train[['REGION_RATING_CLIENT']].values
learn_ver(X=train, y=target )

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(0.07544986804602256, 0.27468139370190797)

In [26]:

train = app_train[['CNT_CHILDREN']].values
learn_ver(X=train, y=target )

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(0.07576433792151291, 0.27525322508830463)

## [Problem 3] Estimation for test data

submit it done

#[Problem 4] Feature quantity engineering


which features to use

CNT_CHILDREN, REGION_RATING_CLIENT, DAYS_ID_PUBLISH, DAYS_BIRTH, SK_ID_CURR

how to pretreat

Analysis of credit information assignment